# User Interaction with our Model
This Notebook a simple pipeline, chaining our submodels and allowing users to interact with our model. The user should be allowed to type in his idea and then get a response including the following information:

1. Listing and Information about similar Businesses
2. A Stock market prediction
3. A success score and how it will perform compared to its competitors (ranking + score)